In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")


In [ ]:
mnist.train.labels[0]

In [ ]:
mnist.train.images[0][251:260]

In [ ]:
for i in range(100):
    plt.figure()
    plt.imshow(-mnist.train.images[i].reshape(28,28),interpolation='nearest', cmap='gray')
    plt.title(mnist.train.labels[i])

In [ ]:
import pywt

In [ ]:
data = mnist.train.images[0].reshape(28,28)
coeffs = pywt.wavedec2(data, 'haar')
for i in range(1,len(coeffs)):
    print i, coeffs[i][0].shape, coeffs[i][1].shape, coeffs[i][2].shape

In [ ]:
def init_coeffs_mnist(coeffs):
    coeffs_zero = [np.zeros((2,2))]
    for h in range(1,5):
        coeffs_zero += [[]]   
        for k in range(3) :
            s = np.shape(coeffs[h][k])        
            coeffs_zero[h] += [np.zeros(s)]        
    return coeffs_zero    

In [ ]:
def add_coeffs_coord(coeffs_ref, coeffs_add, coord):
    v_test = np.array(coeffs_add[0])
    if np.linalg.norm(v_test) < 1e-16:
        coeffs_add[0] = np.copy(coeffs_ref[0])
    for h in range(1,5):
        h_opp = 5 - h
        i = int(coord[0] / 2**h_opp) 
        j = int(coord[1] / 2**h_opp)
        print i, j
        v_test = np.array([coeffs_add[h][0][i,j], coeffs_add[h][1][i,j], coeffs_add[h][2][i,j]])
        if np.linalg.norm(v_test) < 1e-16:
            for k in range(3) :
                coeffs_add[h][k][i,j] = coeffs_ref[h][k][i,j]
            

In [ ]:
def copy_coeffs_mnist(coeffs):
    coeffs_copy = []
    coeffs_copy += [coeffs[0].copy()]
    for h in range(1,5):
        coeffs_copy += [[]]    
        for k in range(3) :
            coeffs_copy[h] += [np.copy(coeffs[h][k])]
    return coeffs_copy    

In [ ]:
def calcule_path(coord, axes):
    path_i = []
    path_j = []
    for h in range(0,5):
        h_opp = 5 - h
        coord_i = int(coord[0] / 2**h_opp) 
        coord_j = int(coord[1] / 2**h_opp)    
        path_i += [axes[h][coord_i]]
        path_j += [axes[h][coord_j]]
    return path_i, path_j

In [ ]:
coeffs_test = copy_coeffs_mnist(coeffs)
coeffs_zero = init_coeffs_mnist(coeffs)
        
plt.figure(figsize=(12,8))
plt.subplot(131)
plt.imshow(-pywt.waverec2(coeffs_test, 'haar'), interpolation='nearest', cmap='gray')

for k in range(3) :
    coeffs_test[4][k] *= 0
    coeffs_zero[4][k] = np.copy(coeffs[4][k])
    
plt.subplot(132)
plt.imshow(-pywt.waverec2(coeffs_test, 'haar'), interpolation='nearest', cmap='gray')

plt.subplot(133)
plt.imshow(-pywt.waverec2(coeffs_zero, 'haar'), interpolation='nearest', cmap='gray')

In [ ]:
axes = []
for h in range(0,5):
    h_opp = 5 - h
    b_moins = -.5
    b_plus = 27.5
    intervalle = b_plus - b_moins
    coord_max = int(26.5 / 2**h_opp) + 1
    unite = max(intervalle / coord_max, 2**h_opp)
    print unite
    coords = []
    for j in range(coord_max):
        coords += [b_moins + (j+.5) * unite]
    axes += [coords]
print axes
print ''

U = []
for h in range(len(axes)):
    #U_ref = {}
    #for pos_i in axes[h]:
    #    for pos_j in axes[h]:
    #        U_ref[h] += [(pos_i, pos_j)]
    U += [{}]
    for i in range(len(axes[h])):
        for j in range(len(axes[h])):
            U[h][(i,j)] = (axes[h][i], axes[h][j])    
    print 'Niveau ', h, ' : '
    print ' U[' , h, '] :', U[h]
    #print ' U_ref[h] :', U_ref[h]
    print ''
    
bsup = [1,2,4,7,14, 0]

def fils_rec(h,i,j):
    if h < 5 and i < bsup[h] and j < bsup[h] :
        rep = [(h,(i,j)), [], [], [], []]
        rep[1] = fils_rec(h+1,i*2, j*2)
        rep[2] = fils_rec(h+1,i*2, j*2 + 1)
        rep[3] = fils_rec(h+1,i*2 + 1, j*2)
        rep[4] = fils_rec(h+1,i*2 + 1, j*2 + 1)
    else:
        rep = []
    return rep
    
U_tree = fils_rec(0,0,0) 

print U_tree


In [ ]:
#coord = (10.5,10.5)
#coord = (8.5,16.5)
#coord = (16.5,4.5)

t = 1

data = mnist.train.images[t].reshape(28,28)
coeffs = pywt.wavedec2(data, 'haar')

coeffs_coord = init_coeffs_mnist(coeffs)

for coord in [(10.5,10.5), (8.5,16.5), (16.5,4.5)]:
    add_coeffs_coord(coeffs, coeffs_coord, coord)

    plt.figure(figsize=(20,12))

    path_i, path_j = calcule_path(coord, axes)

    plt.subplot(131)
    plt.imshow(-pywt.waverec2(coeffs, 'haar'), interpolation='nearest', cmap='gray')
    #for u in U_ref:
    #    plt.plot(u[1],u[0],'+',color='b',markersize=12)
    plt.plot(path_j,path_i,'r')
    plt.plot(path_j,path_i,'r+',markersize=12)
    plt.plot(coord[1],coord[0],'+',color='r',markersize=12)
    plt.xlim([b_moins,b_plus])
    plt.ylim([b_moins,b_plus])
    plt.gca().invert_yaxis()

    plt.subplot(132)
    plt.imshow(-pywt.waverec2(coeffs_coord, 'haar'), interpolation='nearest', cmap='gray')
    plt.plot(path_j,path_i,'r')
    plt.plot(path_j,path_i,'r+',markersize=12)
    plt.plot(coord[1],coord[0],'+',color='r',markersize=12)
    plt.xlim([b_moins,b_plus])
    plt.ylim([b_moins,b_plus])
    plt.gca().invert_yaxis()


## Creation de la base

In [ ]:
len(mnist.train.images)

In [ ]:
B = []
for i in range(len(mnist.train.images)):
    c = mnist.train.labels[i]
    x = mnist.train.images[i].reshape(28,28)
    coeffs = pywt.wavedec2(x, 'haar')
    u0 = (0,0) #U[0][0]
    v0 = np.array([coeffs[0][0][1], coeffs[0][1][0], coeffs[0][1][1]])
    B += [(v0,(c, 0, u0))]
    for h in range(1,5):
        for u in U[h]:
            #v = np.array([coeffs[h][0][u[1]][u[2]], coeffs[h][1][u[1]][u[2]], coeffs[h][2][u[1]][u[2]]])
            v = np.array([coeffs[h][0][u[0]][u[1]], coeffs[h][1][u[0]][u[1]], coeffs[h][2][u[0]][u[1]]])
            #if np.linalg.norm(v) > 1e-16:
            #B += [(v,(c,u))]
            B += [(v,(c,h,u))]            

## Base de test

In [ ]:
B_test = []
for i in range(len(mnist.test.images)):
    c = mnist.test.labels[i]
    x = mnist.test.images[i].reshape(28,28)
    coeffs = pywt.wavedec2(x, 'haar')
    u0 = (0,0) #U[0][0]
    v0 = np.array([coeffs[0][0][1], coeffs[0][1][0], coeffs[0][1][1]])
    B_test += [(v0,(c, 0, u0))]
    for h in range(1,5):
        for u in U[h]:
            #v = np.array([coeffs[h][0][u[1]][u[2]], coeffs[h][1][u[1]][u[2]], coeffs[h][2][u[1]][u[2]]])
            v = np.array([coeffs[h][0][u[0]][u[1]], coeffs[h][1][u[0]][u[1]], coeffs[h][2][u[0]][u[1]]])
            #if np.linalg.norm(v) > 1e-16:
            #B += [(v,(c,u))]
            B_test += [(v,(c,h,u))]            

## Test images simple

In [ ]:
data_visu = [[],[],[],[],[],[],[],[],[],[]]
for i in range(len(mnist.train.images)):
    c = mnist.train.labels[i]
    x = mnist.train.images[i].reshape(28,28)
    data_visu[c] += [x]


In [ ]:
image_moy_moy = np.zeros((28,28))
for c in range(10):
    image_moy = np.mean(np.array(data_visu[c]),0)
    image_moy_moy += image_moy
    plt.figure(figsize=(12,8))
    plt.subplot(131)
    plt.imshow(-image_moy, interpolation='nearest', cmap='gray')
    plt.title(c)
plt.figure(figsize=(12,8))
plt.subplot(131)
plt.imshow(-image_moy_moy, interpolation='nearest', cmap='gray')
plt.title('all')

## Dictionnaire

In [ ]:
Data = [[],[],[],[],[],[],[],[],[],[]]
for c in range(10):
    Data[c] = [{},{},{},{},{}] 
    
for d in B:
    v = d[0]
    c = d[1][0]
    h = d[1][1]
    pos = d[1][2]
    if pos in Data[c][h]:
        Data[c][h][pos] += [v]
    else:
        Data[c][h][pos] = [v]


## Base de test

In [ ]:
Data_test = [[],[],[],[],[],[],[],[],[],[]]
for c in range(10):
    Data_test[c] = [{},{},{},{},{}] 
    
for d in B_test:
    v = d[0]
    c = d[1][0]
    h = d[1][1]
    pos = d[1][2]
    if pos in Data_test[c][h]:
        Data_test[c][h][pos] += [v]
    else:
        Data_test[c][h][pos] = [v]


In [ ]:
print np.mean(Data[0][0][(0,0)],0)
print np.mean(Data[0][1][(0,0)],0)
print np.mean(Data[0][1][(0,1)],0)
print np.mean(Data[0][2][(0,0)],0)
print np.mean(Data[0][2][(0,1)],0)


In [ ]:
c = 1
h = 0
i = 0
j = 0
len(Data[c][h][(i,j)])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111, projection='3d')

h = 0 #4
i = 0 #5
j = 0 #5

test = np.array(Data[0][h][(i,j)])
ax.scatter(test[:,0], test[:,1], test[:,2],c='r', marker = 'o')

test2 = np.array(Data[1][h][(i,j)])
ax.scatter(test2[:,0], test2[:,1], test2[:,2],c='b', marker = '^')

test3 = np.array(Data[3][h][(i,j)])
ax.scatter(test3[:,0], test3[:,1], test3[:,2],c='g', marker = 'd')

ax.view_init(20, -30)

'''for angle in range(0, 360):
    ax.view_init(30, angle)
    plt.draw()
    plt.pause(.001)'''

## Probas elementaires 

In [ ]:
mu = []
Sigma = []
rho = []
for c in range(10):
    mu += [[]]
    Sigma += [[]]
    rho += [[]]
    for h in range(5):
        mu[c] += [{}]
        Sigma[c] += [{}]
        rho[c] += [{}]
        for u in U[h]:
            if u in Data[c][h]:
                data = []
                cpt = 0
                for v in Data[c][h][u]:
                    if np.linalg.norm(v) < 1e-16:
                        cpt += 1
                    else:
                        data += [v]
                if len(data) > 1:
                    mu[c][h][u] = np.mean(Data[c][h][u],0)
                    Sigma[c][h][u] = np.cov(np.array(Data[c][h][u]).T)
                    rho[c][h][u] = float(cpt) / len(Data[c][h][u])
                else:
                    mu[c][h][u] = np.zeros(3)
                    Sigma[c][h][u] = np.zeros((3,3))
                    rho[c][h][u] = 1.
            

In [ ]:
c = 0
h = 0 # 4
u = (0,0) #(12,12)
print mu[c][h][u]
print Sigma[c][h][u]
print rho[c][h][u]


## Reconstruction


In [ ]:
for c in range(10):
    coeffs_zero = init_coeffs_mnist(coeffs)

    coeffs_zero[0][0,1]= mu[c][0][(0,0)][0]
    coeffs_zero[0][1,0] = mu[c][0][(0,0)][1]
    coeffs_zero[0][1,1] = mu[c][0][(0,0)][2]
    for h in range(1,5):
        for u in U[h]:
            coeffs_zero[h][0][u[0]][u[1]] = mu[c][h][u][0]
            coeffs_zero[h][1][u[0]][u[1]] = mu[c][h][u][1]
            coeffs_zero[h][2][u[0]][u[1]] = mu[c][h][u][2]

    plt.figure(figsize=(12,8))
    plt.subplot(131)
    plt.imshow(-pywt.waverec2(coeffs_zero, 'haar'), interpolation='nearest', cmap='gray')



## Test distribution

In [ ]:
fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111, projection='3d')

c = 0
h = 3
u = (5,5)

n = 200

test2 = np.random.multivariate_normal(mu[c][h][u], Sigma[c][h][u] , n)
#test2 = np.random.multivariate_normal(mu[(c,h,u)], Sig_test * 0 , n)
#test2 = np.random.multivariate_normal(-mu[(c,h,u)], np.eye(3,3), n)
ax.scatter(test2[:,0], test2[:,1], test2[:,2],c='b', marker = '^')

#test3 = np.random.multivariate_normal(-mu[(c,h,u)], np.eye(3,3), n)
#ax.scatter(test3[:,0], test3[:,1], test3[:,2],c='g', marker = 'd')

index = np.arange(len(Data[c][h][u]))
np.random.shuffle(index)
test = np.array(Data[c][h][u])
ax.scatter(test[index[:n],0], test[index[:n],1], test[index[:n],2],c='r', marker = 'o')

ax.view_init(20, -30)

## Test proba

In [ ]:
from scipy.stats import multivariate_normal

c_ref = 5
#h = 0
#u = (0,0)

for i in [2]:
    pi = np.ones(10) / 10
    log_score = np.zeros(10)
    mem_pi = [pi]
    for h in range(5):
        print '*************************'
        print '*********' , 'h = ' , h , '************'
        print '*************************'
        for u in U[h]:
            #print U[h]
            #u = (u_ref[1],u_ref[2])
            lik = np.zeros(10)
            for c in range(10):
                v = Data_test[c_ref][h][u][i]
                if np.linalg.norm(v) < 1e-16:
                    lik[c] = rho[c][h][u]
                else:
                    if np.linalg.norm(mu[c][h][u]) > 1e-16:
                        dist = multivariate_normal(mean = mu[c][h][u], cov = Sigma[c][h][u])
                        #print 'mean =', mu[c][h][u]
                        #print 'cov =', Sigma[c][h][u]
                        lik[c] = (1-rho[c][h][u]) * dist.pdf(v)
                    else:
                        lik[c] = 0
                lik[c] = max(lik[c],1e-16)
            print 'lik =' + str(lik) 
            log_score += np.log(lik) 
            max_log_score = max(log_score)
            print 'log_score', log_score
            print 'max(log_score) :', max_log_score
            log_score -= max_log_score
            Z = np.sum(np.exp(log_score))
            pi = np.exp(log_score)/Z
            mem_pi += [pi]
            #for c in range(10):
            #    pi[c] = lik[c]/lik.sum()
            print 'pi =' + str(pi)
            print 'max(pi) = ',max(pi)
            sum_H = - pi * np.log(lik)
            #print np.log(lik)
            print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
            print '**** H = ',sum_H.sum()
    

In [ ]:
plt.figure(figsize = (12,6))
plt.semilogy(mem_pi)
plt.ylim([1e-30,10])
#plt.legend()

In [ ]:
mu[c][h][u]

In [ ]:
np.log(0.06455255)